In [ ]:
import pandas as pd
import networkx as nx

In [ ]:
from iac_sketch import data, etl, sketch, transform

In [ ]:
# DEBUG
import importlib

importlib.reload(data)
importlib.reload(etl)
importlib.reload(transform)
importlib.reload(sketch)
# architect = sketch.Architect("../public/components")
# registry = architect.parse()

In [ ]:
import pandera.pandas as pa

In [ ]:
extract_sys = etl.ExtractSystem()
registry = extract_sys.extract_entities("../public/components/*yaml")
registry

In [ ]:
transform_sys = etl.TransformSystem()
registry = transform_sys.apply_preprocess_transforms(registry)

In [ ]:
registry["component"]

In [ ]:
registry["metadata"]

In [ ]:
# Create a directed graph from the DataFrame
graph = nx.from_pandas_edgelist(
    registry["link"],
    source="source",
    target="target",
    edge_key="link_type",
    create_using=nx.DiGraph,
)

# Visualize the graph
nx.draw(graph, with_labels=True)

In [ ]:
graph.add_nodes_from(registry["metadata"]["entity"].values)

In [ ]:
connected_components = [_ for _ in nx.connected_components(graph.to_undirected())]
metadata = registry["metadata"].set_index("entity")
metadata["connected_component_category"] = -1
for i, comps in enumerate(connected_components):
    metadata.loc[list(comps), "connected_component_category"] = i

In [ ]:
pair_connectivity = nx.all_pairs_node_connectivity(graph)

In [ ]:
sum(pair_connectivity["accept_request"].values()) > 0

In [ ]:
# Find invalid requirements
reqs = registry.view(["requirement", "status", "task", "input"])
is_valid = reqs["status"].isin(["closed", "removed"])
invalid_reqs = reqs.loc[~is_valid]
invalid_reqs

In [ ]:
# Find invalid testcases
registry.view(["testcase", "status"])

In [ ]:
import base64
import io, requests
from IPython.display import Image, display
from PIL import Image as im
import matplotlib.pyplot as plt

In [ ]:
def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    img = im.open(
        io.BytesIO(requests.get("https://mermaid.ink/img/" + base64_string).content)
    )
    plt.imshow(img)
    plt.axis("off")  # allow to hide axis

In [ ]:
graph = """
graph LR;
    A--> B & C & D
    B--> A & E
    C--> A & E
    D--> A
    E--> B & C & D
"""

In [ ]:
graphbytes = graph.encode("utf8")
base64_bytes = base64.urlsafe_b64encode(graphbytes)
base64_string = base64_bytes.decode("ascii")
result = requests.get("https://mermaid.ink/img/" + base64_string).content
parsed_result = io.BytesIO(result)
img = im.open(parsed_result)
img

In [ ]:
mm(
    """
flowchart LR;
    A--> B & C & D
    B--> A & E
    C--> A & E
    D--> A & E
    E--> B & C & D
"""
)